In [ ]:
import cv2
import numpy as np
import sys

In [23]:
from google.colab.patches import cv2_imshow

In [ ]:
pip install opencv-python==3.4.2.16

     |████████████████████████████████| 25.0MB 171kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30


In [ ]:
!git clone https://github.com/pjreddie/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 4.54 MiB/s, done.
Resolving deltas: 100% (3916/3916), done.


In [16]:
# load Google Driver
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
my_path = '/content/drive/My Drive/Colab Notebooks/EVA 5/13.Yolo V2 3 4/'

Mounted at /content/drive


In [17]:
sys.path.append('/content/drive/My Drive/Colab Notebooks/EVA 5/13.Yolo V2 3 4/')

In [5]:
! wget "https://pjreddie.com/media/files/yolov3.weights"

--2020-10-26 14:19:03--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   494KB/s    in 16m 10s 

2020-10-26 14:35:15 (250 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [15]:
# Load Yolo
net = cv2.dnn.readNet("yolov3.weights", "/content/drive/My Drive/Colab Notebooks/EVA 5/13.Yolo V2 3 4/yolov3.cfg")
classes = []
with open("/content/drive/My Drive/Colab Notebooks/EVA 5/13.Yolo V2 3 4/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [18]:
# Loading image
img = cv2.imread("/content/drive/My Drive/Colab Notebooks/EVA 5/13.Yolo V2 3 4/image.jpg")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

In [19]:
# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

In [20]:
# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

In [21]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

In [26]:
font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[i]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2_imshow(img)        
cv2.waitKey(0)
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.